# N-Queens Min-Conflicts — Visualization Notebook

 This notebook:
 - Loads benchmark + solution CSVs
 - Checks for missing configurations / missing seeds
 - Plots small boards (n=10/20/50) with before/after Min-Conflicts
 - Plots large-n structure (scatter + heatmaps)
 - Plots scaling and hyperparameter effects


## 1. Imports, paths, global style


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from itertools import product
from pathlib import Path

# Global Matplotlib style
plt.rcParams.update({
    "figure.dpi": 100,      # overridden per-figure on save
    "font.size": 12,
    "axes.grid": False,
})

# Paths (adjust ROOT if your notebook lives somewhere else)
ROOT = Path("..")
RESULTS_DIR = ROOT / "benchmarks" / "results"
FIG_DIR = ROOT / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

# Export sizing
EXPORT_DPI = 300
FIG_SIZE_SQUARE = (6.67, 6.67)   # 6.67" * 300 dpi ≈ 2000 x 2000 px
FIG_SIZE_WIDE   = (8.0, 4.0)     # ≈ 2400 x 1200 px

## 2. Data loading and grid health check

In [2]:
def load_all_data() -> tuple[pd.DataFrame, dict[int, pd.DataFrame]]:
    """Load benchmark grid and per-n solution CSVs."""
    grid_df = pd.read_csv(RESULTS_DIR / "minconflicts_grid_improved.csv")

    solutions: dict[int, pd.DataFrame] = {}
    for n_value, fname in [
        (10,  "solution_n10.csv"),
        (20,  "solution_n20.csv"),
        (50,  "solution_n50.csv"),
        (1_000_000, "solution_n1e6.csv"),
    ]:
        path = RESULTS_DIR / fname
        if path.exists():
            sol_df = pd.read_csv(path)
            solutions[n_value] = sol_df
        else:
            print(f"[warn] Missing solution file for n={n_value}: {path}")
    return grid_df, solutions


def check_grid_health(
    grid_df: pd.DataFrame,
    expected_seed_count: int | None = 3,
) -> None:
    """
    Print diagnostics about:
      - parameter ranges present
      - missing (n, selector, candidate_count, structured_init) configs
      - configs with < expected_seed_count seeds
    """
    print("=== Parameter ranges present ===")
    print("n values:", sorted(grid_df["n"].unique()))
    print("candidate_selector values:", sorted(grid_df["candidate_selector"].unique()))
    print("candidate_count values:", sorted(grid_df["candidate_count"].unique()))
    print("structured_init values:", sorted(grid_df["structured_init"].unique()))
    print("seed values:", sorted(grid_df["seed"].unique()))
    print()

    param_cols = ["n", "candidate_selector", "candidate_count", "structured_init"]

    # 1) Missing configurations (any combination of params absent at all)
    all_n = sorted(grid_df["n"].unique())
    all_selectors = sorted(grid_df["candidate_selector"].unique())
    all_candidate_counts = sorted(grid_df["candidate_count"].unique())
    all_structured_flags = sorted(grid_df["structured_init"].unique())

    expected_index = pd.MultiIndex.from_product(
        [all_n, all_selectors, all_candidate_counts, all_structured_flags],
        names=param_cols,
    )
    present_index = grid_df.set_index(param_cols).index.unique()

    missing_index = expected_index.difference(present_index)

    if len(missing_index) == 0:
        print("No missing (n, selector, candidate_count, structured_init) configs.")
    else:
        print("Missing configurations (no runs at all for these):")
        print(missing_index.to_frame(index=False).head(20))
        if len(missing_index) > 20:
            print(f"... and {len(missing_index) - 20} more")
    print()

    # 2) Configs with fewer than expected_seed_count seeds
    if expected_seed_count is not None:
        group = (
            grid_df.groupby(param_cols)["seed"]
            .nunique()
            .reset_index(name="seed_count")
        )
        lacking = group[group["seed_count"] < expected_seed_count]

        if lacking.empty:
            print(f"All configs have at least {expected_seed_count} distinct seeds.")
        else:
            print(f"Configs with < {expected_seed_count} seeds:")
            print(lacking.head(20))
            if len(lacking) > 20:
                print(f"... and {len(lacking) - 20} more")
    print()

### Run data load + health check

In [3]:
grid_df, solutions = load_all_data()
print("Grid shape:", grid_df.shape)
check_grid_health(grid_df, expected_seed_count=3)

for n_value, df in solutions.items():
    print(f"Loaded solution for n={n_value}: shape={df.shape}")

Grid shape: (238, 28)
=== Parameter ranges present ===
n values: [np.int64(100), np.int64(1000), np.int64(10000), np.int64(1000000), np.int64(5000000), np.int64(10000000)]
candidate_selector values: ['k_sample', 'nbhd']
candidate_count values: [np.int64(32), np.int64(128), np.int64(512), np.int64(1024), np.int64(2048), np.int64(10240)]
structured_init values: [np.int64(0), np.int64(1)]
seed values: [np.int64(42), np.int64(67), np.int64(123)]

Missing configurations (no runs at all for these):
          n candidate_selector  candidate_count  structured_init
0       100           k_sample            10240                0
1       100           k_sample            10240                1
2       100               nbhd             2048                0
3       100               nbhd             2048                1
4       100               nbhd            10240                0
5       100               nbhd            10240                1
6      1000           k_sample            10240

## 3. Small boards (n=10/20/50) — before/after Min-Conflicts

We reuse one function and pass which row column to use (`row_initial` vs `row_final`).

In [4]:
def plot_small_board_state(
    sol_df: pd.DataFrame,
    row_col: str,
    out_path: Path | None = None,
    *,
    title_suffix: str = "",
    show_attack_rays: bool = True,
    max_rays_queens: int | None = None,
) -> None:
    """
    Visualize one n-queens state (initial or final) on a chess-like board.

    Parameters
    ----------
    sol_df : DataFrame
        Must contain columns: 'n', 'c', row_col, and optionally 'd1', 'd2'.
    row_col : str
        Name of the column to use for queen rows ('row_initial' or 'row_final').
    """
    n = int(sol_df["n"].iloc[0])

    columns = sol_df["c"].values
    rows = sol_df[row_col].values

    fig, axes = plt.subplots(
        1, 2,
        figsize=(10, 5),
        dpi=EXPORT_DPI,
        gridspec_kw={"width_ratios": [2, 1]},
    )
    ax_board, ax_table = axes

    # 1. Checkerboard background
    board_pattern = (np.add.outer(np.arange(n) % 2, np.arange(n) % 2)) % 2
    ax_board.imshow(
        board_pattern,
        cmap="Greys",
        origin="lower",
        extent=(-0.5, n - 0.5, -0.5, n - 0.5),
        alpha=0.35,
    )

    title_core = f"n={n} {title_suffix}".strip()
    ax_board.set_title(title_core or f"n={n} state")
    ax_board.set_xlim(-0.5, n - 0.5)
    ax_board.set_ylim(-0.5, n - 0.5)
    ax_board.set_aspect("equal", adjustable="box")
    ax_board.set_xlabel("Column c")
    ax_board.set_ylabel("Row")

    # Grid lines
    ax_board.set_xticks(np.arange(-0.5, n, 1), minor=True)
    ax_board.set_yticks(np.arange(-0.5, n, 1), minor=True)
    ax_board.grid(which="minor", color="black", linestyle="-", linewidth=0.2, alpha=0.4)
    ax_board.tick_params(which="major", length=0)
    ax_board.set_xticks(range(0, n, max(1, n // 10)))
    ax_board.set_yticks(range(0, n, max(1, n // 10)))

    # 2. Draw queens
    for col, row in zip(columns, rows):
        ax_board.text(
            col,
            row,
            "♛",
            ha="center",
            va="center",
            fontsize=20,
            color="black",
        )

    # 3. Optional attack rays
    if show_attack_rays:
        directions = [
            ( 1,  0), (-1,  0), ( 0,  1), ( 0, -1),
            ( 1,  1), ( 1, -1), (-1,  1), (-1, -1),
        ]

        if max_rays_queens is None:
            ray_indices = list(range(len(columns)))
        else:
            ray_indices = list(range(min(max_rays_queens, len(columns))))

        cmap = plt.get_cmap("tab20")
        num_colors = max(1, len(ray_indices))
        queen_colors = [cmap(i / max(1, num_colors - 1)) for i in range(num_colors)]

        for local_idx, queen_idx in enumerate(ray_indices):
            c0 = int(columns[queen_idx])
            r0 = int(rows[queen_idx])
            q_color = queen_colors[local_idx % num_colors]

            for dc, dr in directions:
                if dc > 0:
                    t_c = (n - 1) - c0
                elif dc < 0:
                    t_c = c0
                else:
                    t_c = np.inf

                if dr > 0:
                    t_r = (n - 1) - r0
                elif dr < 0:
                    t_r = r0
                else:
                    t_r = np.inf

                t_max = min(t_c, t_r)
                if not np.isfinite(t_max) or t_max <= 0:
                    continue

                end_c = c0 + dc * t_max
                end_r = r0 + dr * t_max

                ax_board.plot(
                    [c0, end_c],
                    [r0, end_r],
                    linewidth=3,
                    alpha=0.7,
                    color=q_color,
                )

    # 4. Sample table (use row_final, d1, d2 if present; else row_col only)
    table_cols = ["c"]
    if row_col in sol_df.columns:
        table_cols.append(row_col)
    for extra_col in ["row_final", "d1", "d2"]:
        if extra_col in sol_df.columns and extra_col not in table_cols:
            table_cols.append(extra_col)

    preview = sol_df[table_cols].head(10)
    ax_table.axis("off")
    ax_table.set_title("Sample rows / diagonals")
    tbl = ax_table.table(
        cellText=preview.values,
        colLabels=preview.columns,
        loc="center",
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(8)

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

### Generate small-board figures (before & after for n=10, after for 20/50)

In [5]:
if 10 in solutions:
    sol10 = solutions[10]
    if "row_initial" in sol10.columns:
        plot_small_board_state(
            sol10,
            row_col="row_initial",
            title_suffix="initial state",
            show_attack_rays=True,
            max_rays_queens=10,
            out_path=FIG_DIR / "n10_board_initial.png",
        )
    plot_small_board_state(
        sol10,
        row_col="row_final",
        title_suffix="solution (after Min-Conflicts)",
        show_attack_rays=True,
        max_rays_queens=10,
        out_path=FIG_DIR / "n10_board_final.png",
    )

if 20 in solutions:
    plot_small_board_state(
        solutions[20],
        row_col="row_final",
        title_suffix="solution",
        show_attack_rays=True,
        max_rays_queens=20,
        out_path=FIG_DIR / "n20_board_final.png",
    )

if 50 in solutions:
    plot_small_board_state(
        solutions[50],
        row_col="row_final",
        title_suffix="solution",
        show_attack_rays=True,
        max_rays_queens=50,
        out_path=FIG_DIR / "n50_board_final.png",
    )

## 4. Large n solution (n = 1,000,000): scatter + heatmaps

We keep a scatter view and add three heatmaps:
- density (queen count per bin)
- mean `d1` per bin
- mean move distance per bin

In [6]:
def scatter_large_solution_c_vs_row(
    sol_df: pd.DataFrame,
    color_by: str,
    stride: int,
    out_path: Path | None = None,
) -> None:
    """
    Downsample columns by `stride` and plot c vs row_final with color mapping.
    Axes are in true units (0..n-1); colors use raw values (no normalization).
    """
    n = int(sol_df["n"].iloc[0])

    # stratified downsampling by column
    sample_df = sol_df[sol_df["c"] % stride == 0].copy()

    columns = sample_df["c"].values
    rows = sample_df["row_final"].values
    color_values = sample_df[color_by].values

    fig, ax = plt.subplots(figsize=FIG_SIZE_SQUARE, dpi=EXPORT_DPI)
    scatter = ax.scatter(
        columns,
        rows,
        c=color_values,
        s=1,
        marker="s",
    )
    ax.set_title(f"n={n}: c vs row_final (color={color_by}, stride={stride})")
    ax.set_xlabel("Column c")
    ax.set_ylabel("Row")
    ax.set_xlim(0, n - 1)
    ax.set_ylim(0, n - 1)
    ax.invert_yaxis()

    colorbar = fig.colorbar(scatter, ax=ax)
    colorbar.set_label(color_by)

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)


def compute_heatmap_c_vs_row(
    sol_df: pd.DataFrame,
    value_col: str | None,
    bins: int,
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Aggregate all queens into a 2D histogram over (c, row_final).

    - If value_col is None: return counts per bin.
    - Else: return mean of value_col per bin (using counts for averaging).
    """
    n = int(sol_df["n"].iloc[0])
    columns = sol_df["c"].values.astype(float)
    rows = sol_df["row_final"].values.astype(float)

    # Bin edges from -0.5 .. n-0.5 so bin centers align with square centers
    x_edges = np.linspace(-0.5, n - 0.5, bins + 1)
    y_edges = np.linspace(-0.5, n - 0.5, bins + 1)

    if value_col is None:
        H, _, _ = np.histogram2d(columns, rows, bins=[x_edges, y_edges])
        return H.T, x_edges, y_edges

    values = sol_df[value_col].values.astype(float)
    H_sum, _, _ = np.histogram2d(
        columns, rows, bins=[x_edges, y_edges], weights=values
    )
    H_count, _, _ = np.histogram2d(columns, rows, bins=[x_edges, y_edges])
    H_mean = H_sum / np.maximum(H_count, 1.0)  # avoid divide-by-zero
    return H_mean.T, x_edges, y_edges


def plot_heatmap_c_vs_row(
    heatmap: np.ndarray,
    x_edges: np.ndarray,
    y_edges: np.ndarray,
    *,
    title: str,
    colorbar_label: str,
    out_path: Path | None = None,
    cmap: str = "viridis",
) -> None:
    """Render a precomputed heatmap over (c, row_final)."""
    fig, ax = plt.subplots(figsize=FIG_SIZE_SQUARE, dpi=EXPORT_DPI)

    # extent uses actual board coordinates
    extent = [x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]]

    image = ax.imshow(
        heatmap,
        origin="lower",
        extent=extent,
        aspect="equal",
        cmap=cmap,
    )
    ax.set_xlabel("Column c")
    ax.set_ylabel("Row")
    ax.set_title(title)

    cbar = fig.colorbar(image, ax=ax)
    cbar.set_label(colorbar_label)

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

### Generate scatter + three heatmaps for n = 1,000,000

In [7]:
if 1_000_000 in solutions:
    sol1m = solutions[1_000_000]

    # Scatter views (still useful for intuition)
    scatter_large_solution_c_vs_row(
        sol1m,
        color_by="d1",
        stride=100,
        out_path=FIG_DIR / "n1e6_scatter_c_vs_row_d1.png",
    )
    scatter_large_solution_c_vs_row(
        sol1m,
        color_by="move_distance",
        stride=100,
        out_path=FIG_DIR / "n1e6_scatter_c_vs_row_move_distance.png",
    )

    # Heatmap 1: density
    H_density, x_edges, y_edges = compute_heatmap_c_vs_row(
        sol1m,
        value_col=None,
        bins=1000,
    )
    plot_heatmap_c_vs_row(
        H_density,
        x_edges,
        y_edges,
        title="n=1,000,000: queen density in (c, row_final)",
        colorbar_label="queen count per bin",
        out_path=FIG_DIR / "n1e6_heatmap_density.png",
        cmap="magma",
    )

    # Heatmap 2: mean d1
    H_d1, x_edges, y_edges = compute_heatmap_c_vs_row(
        sol1m,
        value_col="d1",
        bins=1000,
    )
    plot_heatmap_c_vs_row(
        H_d1,
        x_edges,
        y_edges,
        title="n=1,000,000: mean d1 per bin",
        colorbar_label="mean d1 (r - c)",
        out_path=FIG_DIR / "n1e6_heatmap_mean_d1.png",
        cmap="coolwarm",
    )

    # Heatmap 3: mean move_distance
    H_move, x_edges, y_edges = compute_heatmap_c_vs_row(
        sol1m,
        value_col="move_distance",
        bins=1000,
    )
    plot_heatmap_c_vs_row(
        H_move,
        x_edges,
        y_edges,
        title="n=1,000,000: mean move distance per bin",
        colorbar_label="mean |row_final - row_initial|",
        out_path=FIG_DIR / "n1e6_heatmap_mean_move_distance.png",
        cmap="viridis",
    )

## 5. Scaling of the best configuration

We summarise runs per `n` for a fixed hyperparameter configuration and
plot steps + runtime vs n

In [8]:
def summarize_by_n_for_config(
    grid_df: pd.DataFrame,
    candidate_selector_value: str,
    candidate_count_value: int,
    structured_init_value: int,
) -> pd.DataFrame:
    """Aggregate mean/std of steps & time per n for one hyperparameter setting."""
    subset = grid_df[
        (grid_df["candidate_selector"] == candidate_selector_value)
        & (grid_df["candidate_count"] == candidate_count_value)
        & (grid_df["structured_init"] == structured_init_value)
        & (grid_df["solved"] == 1)
    ].copy()

    summary = (
        subset.groupby("n")
        .agg(
            steps_mean=("steps", "mean"),
            steps_std=("steps", "std"),
            time_mean=("total_time_sec", "mean"),
            time_std=("total_time_sec", "std"),
            solved_runs=("solved", "count"),
        )
        .reset_index()
    )
    return summary


def plot_scaling_for_config(
    summary_df: pd.DataFrame,
    label: str,
    out_path: Path | None = None,
) -> None:
    """Steps and runtime vs n on log–log scales, with data labels."""
    fig, ax_left = plt.subplots(figsize=FIG_SIZE_WIDE, dpi=EXPORT_DPI)

    n_values = summary_df["n"].values
    steps_mean = summary_df["steps_mean"].values
    steps_std = summary_df["steps_std"].values
    time_mean = summary_df["time_mean"].values
    time_std = summary_df["time_std"].values

    def fmt_steps(y: float) -> str:
        return f"{int(round(y)):,}"

    def fmt_time(y: float) -> str:
        return f"{y:.1f}" if y >= 1 else f"{y:.2f}"

    # Steps vs n
    ax_left.errorbar(
        n_values,
        steps_mean,
        yerr=steps_std,
        fmt="o-",
        label="steps (mean ± std)",
    )
    ax_left.set_xscale("log")
    ax_left.set_yscale("log")
    ax_left.set_xlabel("n (log scale)")
    ax_left.set_ylabel("steps")

    counter = 0
    offset_options = [(-10, 8), (-10, 15), (-12, 8), (-20, 8),(-50, 0)]
    for x, y in zip(n_values, steps_mean):
        ax_left.annotate(
            fmt_steps(y),
            xy=(x, y),
            xytext=offset_options[counter],
            textcoords="offset points",
            fontsize=8,
        )
        counter += 1

    # Runtime on right axis
    ax_right = ax_left.twinx()
    ax_right.errorbar(
        n_values,
        time_mean,
        yerr=time_std,
        fmt="s--",
        label="runtime (s, mean ± std)",
        color="tab:orange",
    )
    ax_right.set_yscale("log")
    ax_right.set_ylabel("runtime (seconds)")

    counter = 0
    offset_options = [(5, -5), (5, -15), (5, -15), (5, -15),(-8, -20)]
    for x, y in zip(n_values, time_mean):
        ax_right.annotate(
            fmt_time(y),
            xy=(x, y),
            xytext=offset_options[counter],
            textcoords="offset points",
            fontsize=8,
            color="tab:orange",
        )
        counter += 1

    # Combined legend
    lines_left, labels_left = ax_left.get_legend_handles_labels()
    lines_right, labels_right = ax_right.get_legend_handles_labels()
    ax_right.legend(
        lines_left + lines_right,
        labels_left + labels_right,
        loc="upper left",
    )

    fig.suptitle(label)

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

### Generate scaling plot for best configuration (k-sampling, 2048 candidates, unstructured init)

In [9]:
best_summary = summarize_by_n_for_config(
    grid_df,
    candidate_selector_value="k_sample",
    candidate_count_value=2048,
    structured_init_value=0,
)
plot_scaling_for_config(
    best_summary,
    label="Scaling for k-sampling (2048 candidates, unstructured init)",
    out_path=FIG_DIR / "scaling_k_sample_2048_unstructured.png",
)

## 6. Hyperparameter effects at n = 1,000,000

We show how the mean number of steps depends on:
- candidate_count for each selector (k-sampling vs neighborhood)
- candidate_count for structured vs unstructured init (for k-sampling)

In [10]:
def summarize_at_fixed_n(grid_df: pd.DataFrame, n_fixed: int) -> pd.DataFrame:
    """Aggregate mean/std over seeds for each hyperparameter setting at given n."""
    subset = grid_df[(grid_df["n"] == n_fixed) & (grid_df["solved"] == 1)].copy()
    summary = (
        subset.groupby(["candidate_selector", "candidate_count", "structured_init"])
        .agg(
            steps_mean=("steps", "mean"),
            steps_std=("steps", "std"),
            time_mean=("total_time_sec", "mean"),
            time_std=("total_time_sec", "std"),
        )
        .reset_index()
    )
    return summary


def _pretty_selector_label(raw_selector: str) -> str:
    """Map raw selector string to a nicer label for plots."""
    if raw_selector == "k_sample":
        return "k-sampling"
    if raw_selector == "nbhd":
        return "neighborhood search"
    return raw_selector


def plot_steps_vs_candidate_count_by_selector(
    summary_df: pd.DataFrame,
    n_fixed: int,
    structured_init_value: int,
    out_path: Path | None = None,
) -> None:
    """
    One line per candidate_selector (k-sampling vs neighborhood) for fixed init flag.
    """
    fig, ax = plt.subplots(figsize=FIG_SIZE_WIDE, dpi=EXPORT_DPI)

    for selector_value in sorted(summary_df["candidate_selector"].unique()):
        subset = summary_df[
            (summary_df["candidate_selector"] == selector_value)
            & (summary_df["structured_init"] == structured_init_value)
        ]
        if subset.empty:
            continue

        candidate_counts = subset["candidate_count"].values
        steps_mean = subset["steps_mean"].values
        steps_std = subset["steps_std"].values

        label = _pretty_selector_label(selector_value)
        ax.errorbar(
            candidate_counts,
            steps_mean,
            yerr=steps_std,
            marker="o",
            linestyle="-",
            label=label,
        )

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("candidate_count")
    ax.set_ylabel("steps (mean ± std)")
    ax.set_title(f"n={n_fixed}: candidate_count vs steps (structured_init={structured_init_value})")
    ax.legend()

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)


def plot_steps_vs_candidate_count_by_init(
    summary_df: pd.DataFrame,
    n_fixed: int,
    selector_value: str,
    out_path: Path | None = None,
) -> None:
    """
    One line per structured_init flag (0 vs 1) for a fixed selector (e.g. k-sampling).
    """
    fig, ax = plt.subplots(figsize=FIG_SIZE_WIDE, dpi=EXPORT_DPI)

    for structured_flag in [0, 1]:
        subset = summary_df[
            (summary_df["candidate_selector"] == selector_value)
            & (summary_df["structured_init"] == structured_flag)
        ]
        if subset.empty:
            continue

        candidate_counts = subset["candidate_count"].values
        steps_mean = subset["steps_mean"].values
        steps_std = subset["steps_std"].values

        label = f"{_pretty_selector_label(selector_value)}, structured_init={structured_flag}"
        ax.errorbar(
            candidate_counts,
            steps_mean,
            yerr=steps_std,
            marker="o",
            linestyle="-",
            label=label,
        )

    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("candidate_count")
    ax.set_ylabel("steps (mean ± std)")
    ax.set_title(f"n={n_fixed}: candidate_count vs steps (selector={_pretty_selector_label(selector_value)})")
    ax.legend()

    if out_path is not None:
        fig.tight_layout()
        fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

### Generate hyperparameter effect plots at n = 1,000,000

In [11]:
n_fixed = 1_000_000
summary_1m = summarize_at_fixed_n(grid_df, n_fixed=n_fixed)

plot_steps_vs_candidate_count_by_selector(
    summary_1m,
    n_fixed=n_fixed,
    structured_init_value=0,
    out_path=FIG_DIR / "n1e6_candidates_vs_steps_by_selector_unstructured.png",
)

plot_steps_vs_candidate_count_by_init(
    summary_1m,
    n_fixed=n_fixed,
    selector_value="k_sample",
    out_path=FIG_DIR / "n1e6_candidates_vs_steps_k_sample_by_init.png",
)

In [12]:
# Cell 2: utilities to bin the big-n solution

def compute_binned_stats(
    sol_df: pd.DataFrame,
    num_bins: int = 800,
) -> dict:
    """
    Aggregate an n-queens solution into a coarse 2D grid of bins.

    Inputs
    ------
    sol_df : DataFrame with columns:
        - 'n' (single value)
        - 'c' (column index)
        - 'row_final'
        - 'row_initial'
        - 'd1'  (r - c)
        - optionally 'd2' etc. (not needed here)
    num_bins : int
        Number of bins in each dimension (bins x bins).

    Returns
    -------
    stats : dict with keys:
        - 'n'
        - 'num_bins'
        - 'bin_size_board'  (approx squares per side per bin)
        - 'density'         (counts per (c,row) bin)
        - 'mean_d1'
        - 'mean_move_dist'
        - 'x_edges', 'y_edges'  (bin edges in board coordinates)
    """
    n = int(sol_df["n"].iloc[0])

    cols = sol_df["c"].to_numpy(dtype=np.float64)
    rows = sol_df["row_final"].to_numpy(dtype=np.float64)
    d1_vals = sol_df["d1"].to_numpy(dtype=np.float64)
    move_dist = np.abs(
        sol_df["row_final"].to_numpy(dtype=np.float64)
        - sol_df["row_initial"].to_numpy(dtype=np.float64)
    )

    # Histogram 2D for density
    density, x_edges, y_edges = np.histogram2d(
        cols, rows,
        bins=[num_bins, num_bins],
        range=[[0, n], [0, n]],
    )
    # Note: density.shape == (num_bins, num_bins)

    # Weighted sums for d1 and move_distance
    d1_sum, _, _ = np.histogram2d(
        cols, rows,
        bins=[num_bins, num_bins],
        range=[[0, n], [0, n]],
        weights=d1_vals,
    )
    move_sum, _, _ = np.histogram2d(
        cols, rows,
        bins=[num_bins, num_bins],
        range=[[0, n], [0, n]],
        weights=move_dist,
    )

    # Avoid division by zero
    safe_density = np.where(density > 0, density, np.nan)

    mean_d1 = d1_sum / safe_density
    mean_move = move_sum / safe_density

    # Replace NaNs (empty bins) with 0 for plotting
    mean_d1 = np.nan_to_num(mean_d1, nan=0.0)
    mean_move = np.nan_to_num(mean_move, nan=0.0)

    bin_size_board = n / num_bins  # how many board squares per bin (one side)

    return {
        "n": n,
        "num_bins": num_bins,
        "bin_size_board": bin_size_board,
        "density": density,
        "mean_d1": mean_d1,
        "mean_move_dist": mean_move,
        "x_edges": x_edges,
        "y_edges": y_edges,
    }


In [13]:
# Cell 3: generic heatmap helper with bin-size annotation

def plot_heatmap_with_bin_info(
    grid: np.ndarray,
    n: int,
    num_bins: int,
    bin_size_board: float,
    title: str,
    colorbar_label: str,
    cmap: str = "viridis",
    out_path: Path | None = None,
) -> None:
    """
    Plot a square heatmap in (c, row) coordinates, and annotate
    the bin size at the bottom of the figure.
    """
    fig, ax = plt.subplots(figsize=FIG_SIZE_SQUARE, dpi=EXPORT_DPI)

    # Show in board coordinates: x in [0, n], y in [0, n]
    im = ax.imshow(
        grid.T,                # transpose so rows go up
        origin="lower",
        extent=[0, n, 0, n],
        aspect="equal",
        cmap=cmap,
        interpolation="nearest",
    )

    ax.set_title(title)
    ax.set_xlabel("Column c")
    ax.set_ylabel("Row")

    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label(colorbar_label)

    # Text describing how much aggregation happened
    fig.text(
        0.5, 0.02,
        f"{num_bins} × {num_bins} bins  "
        f"(each bin ≈ {bin_size_board:.0f} × {bin_size_board:.0f} board squares)",
        ha="center",
        va="center",
        fontsize=9,
    )

    fig.tight_layout(rect=[0, 0.04, 1, 1])  # leave room for bottom text

    if out_path is not None:
        out_path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(out_path, bbox_inches="tight")

    plt.close(fig)


In [14]:
# Cell 4: convenience wrapper to plot all three heatmaps for a solution

def plot_solution_heatmaps(
    sol_df: pd.DataFrame,
    num_bins: int = 800,
    out_dir: Path | None = None,
    prefix: str = "n1e6",
) -> None:
    """
    Produce three heatmaps:
      1) queen density per bin
      2) mean d1 per bin
      3) mean move distance per bin
    """
    stats = compute_binned_stats(sol_df, num_bins=num_bins)

    n = stats["n"]
    grid_density = stats["density"]
    grid_mean_d1 = stats["mean_d1"]
    grid_mean_move = stats["mean_move_dist"]
    bin_size_board = stats["bin_size_board"]

    if out_dir is None:
        out_dir = Path("figures")

    # 1) density
    plot_heatmap_with_bin_info(
        grid_density,
        n=n,
        num_bins=num_bins,
        bin_size_board=bin_size_board,
        title=f"n={n:,}: queen density in (c, row_final)",
        colorbar_label="queen count per bin",
        cmap="magma",
        out_path=out_dir / f"{prefix}_density_heatmap.png",
    )

    # 2) mean d1
    plot_heatmap_with_bin_info(
        grid_mean_d1,
        n=n,
        num_bins=num_bins,
        bin_size_board=bin_size_board,
        title=f"n={n:,}: mean d1 = (row - col) per bin",
        colorbar_label="mean d1",
        cmap="coolwarm",
        out_path=out_dir / f"{prefix}_mean_d1_heatmap.png",
    )

    # 3) mean move distance
    plot_heatmap_with_bin_info(
        grid_mean_move,
        n=n,
        num_bins=num_bins,
        bin_size_board=bin_size_board,
        title=f"n={n:,}: mean |row_final - row_initial| per bin",
        colorbar_label="mean move distance",
        cmap="viridis",
        out_path=out_dir / f"{prefix}_mean_move_heatmap.png",
    )


In [15]:
# Cell 5: density profiles + histogram

def plot_density_profiles_and_histogram(
    stats: dict,
    out_dir: Path | None = None,
    prefix: str = "n1e6",
) -> None:
    """
    Given the stats from compute_binned_stats, quantify density uniformity:
      - row-profile (sum over columns)
      - column-profile (sum over rows)
      - histogram of per-bin densities
    """
    n = stats["n"]
    density = stats["density"]
    num_bins = stats["num_bins"]

    if out_dir is None:
        out_dir = Path("figures")

    # Row/column profiles
    # density.shape = (num_bins, num_bins) with axes: [x_bins, y_bins]
    # We want:
    #   - column index along x -> sum over rows
    #   - row index along y -> sum over columns
    col_profile = density.sum(axis=1)   # sum over rows
    row_profile = density.sum(axis=0)   # sum over cols

    # Bin centers in board coordinates
    bin_size = n / num_bins
    col_centers = (np.arange(num_bins) + 0.5) * bin_size
    row_centers = (np.arange(num_bins) + 0.5) * bin_size

    # Ideal uniform value (one queen per column and per row in total).
    # Total queens = n; total bins per axis = num_bins.
    # Expectation per 1D bin ≈ n / num_bins.
    ideal_1d = n / num_bins

    # --- Profiles figure ---
    fig, axes = plt.subplots(1, 2, figsize=FIG_SIZE_WIDE, dpi=EXPORT_DPI)

    ax_col, ax_row = axes

    ax_col.plot(col_centers, col_profile, linewidth=1)
    ax_col.axhline(ideal_1d, color="red", linestyle="--", linewidth=0.8, label="expected")
    ax_col.set_title("Column density profile")
    ax_col.set_xlabel("Column (board units)")
    ax_col.set_ylabel("queens per column-bin")
    ax_col.legend()

    ax_row.plot(row_centers, row_profile, linewidth=1)
    ax_row.axhline(ideal_1d, color="red", linestyle="--", linewidth=0.8, label="expected")
    ax_row.set_title("Row density profile")
    ax_row.set_xlabel("Row (board units)")
    ax_row.set_ylabel("queens per row-bin")
    ax_row.legend()

    fig.suptitle(f"n={n:,}: 1D density profiles")
    fig.tight_layout(rect=[0, 0, 1, 0.95])

    fig_path_profiles = out_dir / f"{prefix}_density_profiles.png"
    fig.savefig(fig_path_profiles, bbox_inches="tight")
    plt.close(fig)

    # --- Histogram of 2D bin densities ---
    flat_density = density.ravel()
    fig, ax = plt.subplots(figsize=FIG_SIZE_WIDE, dpi=EXPORT_DPI)

    ax.hist(flat_density, bins=30, edgecolor="black")
    ax.set_title(f"n={n:,}: histogram of queen counts per 2D bin")
    ax.set_xlabel("queens per bin")
    ax.set_ylabel("number of bins")

    mean_val = flat_density.mean()
    std_val = flat_density.std()
    ax.axvline(mean_val, color="red", linestyle="--", label=f"mean={mean_val:.2f}")
    ax.legend()

    fig.tight_layout()
    fig_path_hist = out_dir / f"{prefix}_density_histogram.png"
    fig.savefig(fig_path_hist, bbox_inches="tight")
    plt.close(fig)

    print(f"[saved] {fig_path_profiles}")
    print(f"[saved] {fig_path_hist}")


In [16]:
# Cell 6: gradient magnitude heatmap (optional)

def plot_density_gradient_heatmap(
    stats: dict,
    out_dir: Path | None = None,
    prefix: str = "n1e6",
) -> None:
    """
    Visualize how quickly density changes across the board by plotting
    |∇density| (gradient magnitude) as a heatmap.
    """
    n = stats["n"]
    density = stats["density"]
    num_bins = stats["num_bins"]
    bin_size_board = stats["bin_size_board"]

    if out_dir is None:
        out_dir = Path("figures")

    # Discrete gradient
    grad_x, grad_y = np.gradient(density)
    grad_mag = np.sqrt(grad_x**2 + grad_y**2)

    fig, ax = plt.subplots(figsize=FIG_SIZE_SQUARE, dpi=EXPORT_DPI)
    im = ax.imshow(
        grad_mag.T,
        origin="lower",
        extent=[0, n, 0, n],
        aspect="equal",
        cmap="plasma",
        interpolation="nearest",
    )

    ax.set_title(f"n={n:,}: |∇density| per bin")
    ax.set_xlabel("Column c")
    ax.set_ylabel("Row")

    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label("gradient magnitude of density")

    fig.text(
        0.5, 0.02,
        f"{num_bins} × {num_bins} bins  "
        f"(each bin ≈ {bin_size_board:.0f} × {bin_size_board:.0f} board squares)",
        ha="center",
        va="center",
        fontsize=9,
    )

    fig.tight_layout(rect=[0, 0.04, 1, 1])
    out_path = out_dir / f"{prefix}_density_gradient_heatmap.png"
    fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

    print(f"[saved] {out_path}")


In [17]:
sol_df_1e6 = solutions[1000000]
stats_1e6 = compute_binned_stats(sol_df_1e6, num_bins=1000)
plot_solution_heatmaps(sol_df_1e6, num_bins=1000, out_dir=FIG_DIR, prefix="n1e6")
plot_density_profiles_and_histogram(stats_1e6, out_dir=FIG_DIR, prefix="n1e6")
plot_density_gradient_heatmap(stats_1e6, out_dir=FIG_DIR, prefix="n1e6")


[saved] ..\figures\n1e6_density_profiles.png
[saved] ..\figures\n1e6_density_histogram.png
[saved] ..\figures\n1e6_density_gradient_heatmap.png
